# Plasma Coupling Using COMSOL Results

In this example, we use a COMSOL front-face coupling calculation provided by ORNL, exported as a standard Touchstone file.

The Touchstone file is first import as a scikit-rf Network, which is then modified to fit the WEST ICRH antenna electrical model requirements.

In [25]:
import numpy as np
import skrf as rf

# WEST ICRH Antenna package
import sys; sys.path.append('..')
from west_ic_antenna import WestIcrhAntenna

In [26]:
front_face_conventional = rf.Network(
    '../west_ic_antenna/data/Sparameters/front_faces/COMSOL/ORNL_front_face_conventional.s4p')
print(front_face_conventional)  # 50 Ohm S-param component at a single frequency of 55 MHz

4-Port Network: 'ORNL_front_face_conventional',  55000000.0-55000000.0 Hz, 1 pts, z0=[50.+0.j 50.+0.j 50.+0.j 50.+0.j]


The ports have been defined as:
<img src="COMSOL_WEST_port_index.png" />


So before to use the S-parameters directly to feed the electrical model, we need to:
- deembed the ports by 0.3m.
- renomalize port reference impedance to the front-face coax characteristic impedances. 
- reverse ports 2 and 3.

In [27]:
# creating a 50 Ohm dummy coax line to be removed from the front face 
media_coax = rf.DefinedGammaZ0(frequency=front_face_conventional.frequency) # 50 Ohm TEM media
extra_line = media_coax.line(d=0.3, unit='m')
# deembedding all the 4 pourts
for port_idx in range(4):
    front_face_conventional = rf.connect(front_face_conventional, port_idx, extra_line.inv, 0)

We expect the port to have a characteristic impedance of about 46.64 ohm, so we renormalize the Network to fit this need:

In [28]:
front_face_conventional.renormalize(46.64)  # done inplace

And finally, for historical reasons (may change in a near future ;), the S-matrix port ordering should be ajusted:

In [29]:
front_face_conventional.renumber([1, 2], [2, 1]) # done inplace

OK, so now we can create the WEST antenna object:

In [30]:
ant = WestIcrhAntenna(front_face=front_face_conventional,
                     frequency=front_face_conventional.frequency) # restrict to single frequ

Let's match the antenna for this coupling:

In [31]:
Cs = ant.match_both_sides(f_match=55e6)

Looking for individual solutions separately for 1st guess...
Wrong solution (out of range capacitor) ! Re-doing...
False solution #1: [150. 150.]
True solution #1: [52.57986227 45.88696785]
True solution #1: [52.30894839 46.0700872 ]
Searching for the active match point solution...
Reducing search range to +/- 5pF around individual solutions
True solution #1: [53.67807308 46.12207788 53.62800637 46.30935881]


The coupling resistance of the antenna for this coupling in a nominal dipole excitation is:

In [32]:
power = [1, 1]
phase = [0, np.pi]

# Coupling resistance
ant.Rc(power, phase)

array([0.70081638, 0.6878438 ])

The voltage and currents are:

In [44]:
power = [1, 1]  # MW, to adjust to fit with experiment
phase = [0, np.pi]  # rad

abs(ant.voltages(power, phase))  # results in kV

array([[22.16126386, 23.75535708, 20.56902439, 25.21223565]])

In [41]:
abs(ant.currents(power, phase))  # results in kA

array([[0.70248687, 0.75556107, 0.65298786, 0.80405425]])